In [37]:
import pandas as pd
from textblob import TextBlob
import nltk
import re
import numpy as np

Importo el archivo y analisó su formato

In [3]:
data = pd.read_csv('Datasets/user_reviews.csv')

In [4]:
data.shape

(59333, 9)

In [5]:
data

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted November 5, 2011.",NaN,1250.0,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted July 15, 2011.",NaN,22200.0,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,NaN,"Posted April 21, 2011.",NaN,43110.0,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,NaN,"Posted June 24, 2014.",NaN,251610.0,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,NaN,"Posted September 8, 2013.",NaN,227300.0,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59328,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,Posted July 10.,NaN,70.0,No ratings yet,True,a must have classic from steam definitely wort...
59329,76561198312638244,http://steamcommunity.com/profiles/76561198312...,NaN,Posted July 8.,NaN,362890.0,No ratings yet,True,this game is a perfect remake of the original ...
59330,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,1 person found this review funny,Posted July 3.,NaN,273110.0,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59331,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,NaN,Posted July 20.,NaN,730.0,No ratings yet,True,:D


Tranformo los dados de la columna 'review' en strings

In [6]:
data.review = data.review.astype(str)

Creo una función para analisis de sentimiento

In [17]:
def sentiment_analysis(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        polaridad = sentiment
        sentimento = 2
        significado = 'Positivo'
    elif sentiment < 0:
        polaridad = sentiment
        sentimento = 0
        significado = 'Negativo'
    else:
        polaridad = sentiment
        sentimento = 1
        significado = 'Neutro'
    
    return polaridad, sentimento, significado

Creo un dataframe con la columna 'reviews'

In [8]:
df = pd.DataFrame(data.review)

In [9]:
df

,review
0,Simple yet with great replayability. In my opi...
1,It's unique and worth a playthrough.
2,Great atmosphere. The gunplay can be a bit chu...
3,I know what you think when you see this title ...
4,For a simple (it's actually not all that simpl...
...,...
59328,a must have classic from steam definitely wort...
59329,this game is a perfect remake of the original ...
59330,had so much fun plaing this and collecting res...
59331,:D


Aplico la funcion de analisis de sentimiento en cada linea del dataframe, pero me retorna un error porque el resultado de la funcion no tiene el mismo tamaño que la cantidad de coluna que quiero crear

In [18]:
df[['Polaridad', 'Sentimiento', 'Significado']] = df.review.apply(sentiment_analysis)

ValueError: Columns must be same length as key

Corrijo la función para que me retorne una serie con las columnas e aplico la función a cada linea con el metodo .apply()

In [19]:
def sentiment_analysis(text):
    blob = TextBlob(text)
    sentiment = blob.sentiment.polarity
    if sentiment > 0:
        polaridad = sentiment
        sentimento = 2
        significado = 'Positivo'
    elif sentiment < 0:
        polaridad = sentiment
        sentimento = 0
        significado = 'Negativo'
    else:
        polaridad = sentiment
        sentimento = 1
        significado = 'Neutro'
    
    return pd.Series([polaridad, sentimento, significado])

In [20]:
df[['polaridad', 'sentimiento', 'significado']] = df.review.apply(sentiment_analysis)

In [21]:
df[df['significado'] == 'Positivo']

,review,polaridad,sentimiento,significado
0,Simple yet with great replayability. In my opi...,0.174444,2,Positivo
1,It's unique and worth a playthrough.,0.337500,2,Positivo
2,Great atmosphere. The gunplay can be a bit chu...,0.050000,2,Positivo
3,I know what you think when you see this title ...,0.114583,2,Positivo
6,A suitably punishing roguelike platformer. Wi...,0.231548,2,Positivo
...,...,...,...,...
59328,a must have classic from steam definitely wort...,0.233333,2,Positivo
59329,this game is a perfect remake of the original ...,0.142500,2,Positivo
59330,had so much fun plaing this and collecting res...,0.087500,2,Positivo
59331,:D,1.000000,2,Positivo


In [22]:
df[df['significado'] == 'Negativo']

,review,polaridad,sentimiento,significado
4,For a simple (it's actually not all that simpl...,-0.036111,0,Negativo
5,Very fun little game to play when your bored o...,-0.099500,0,Negativo
9,"Random drops and random quests, with stat poin...",-0.333333,0,Negativo
16,The ending to this game is.... ♥♥♥♥♥♥♥.... Jus...,-0.333333,0,Negativo
22,This game is Marvellous.,-0.400000,0,Negativo
...,...,...,...,...
59309,GET THIS GAME AND CHAPTER TWO!!!!!!!!!!!!!!!!!...,-1.000000,0,Negativo
59310,THIS GAME!!!!!!!!!!!!!!!!!! WOOOOOOOOOOOOOOOOO...,-1.000000,0,Negativo
59311,DON'T INSTALL THIS GAME!!!!!!!!!!!!!!!!!!!!!!!...,-0.333333,0,Negativo
59312,"Game is terrible! So, Iv tried 15hrs tryin to ...",-0.700000,0,Negativo


In [23]:
df[df['significado'] == 'Neutro']

,review,polaridad,sentimiento,significado
18,Git gud,0.0,1,Neutro
24,This game is fantastic if you are looking to D...,0.0,1,Neutro
27,ZIKA DO BAILE,0.0,1,Neutro
30,10/10 would eat your money for hats and keys,0.0,1,Neutro
32,mt bom,0.0,1,Neutro
...,...,...,...,...
59308,Hi I don't know what this is,0.0,1,Neutro
59313,"uplay, everytime",0.0,1,Neutro
59320,dont ask,0.0,1,Neutro
59324,Gra naprawdę fajna.Ale jest kilka rzeczy do kt...,0.0,1,Neutro


Incluo las columnas 'user_id' e 'item_id' e guardo el dataframe en un archivo csv

In [68]:
df[['user_id', 'item_id']] = data[['user_id', 'item_id']]

In [73]:
df

,review,polaridad,sentimiento,significado,user_id,item_id
0,Simple yet with great replayability. In my opi...,0.174444,2,Positivo,76561197970982479,1250.0
1,It's unique and worth a playthrough.,0.337500,2,Positivo,76561197970982479,22200.0
2,Great atmosphere. The gunplay can be a bit chu...,0.050000,2,Positivo,76561197970982479,43110.0
3,I know what you think when you see this title ...,0.114583,2,Positivo,js41637,251610.0
4,For a simple (it's actually not all that simpl...,-0.036111,0,Negativo,js41637,227300.0
...,...,...,...,...,...,...
59328,a must have classic from steam definitely wort...,0.233333,2,Positivo,76561198312638244,70.0
59329,this game is a perfect remake of the original ...,0.142500,2,Positivo,76561198312638244,362890.0
59330,had so much fun plaing this and collecting res...,0.087500,2,Positivo,LydiaMorley,273110.0
59331,:D,1.000000,2,Positivo,LydiaMorley,730.0


In [74]:
df.to_csv('Datasets/sentiment_analysis_reviews.csv', index= False)